### Data Ingestion Module

In [3]:
import os
%pwd

'c:\\Users\\dipgh\\OneDrive\\Desktop\\MLOps\\TextSummarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\dipgh\\OneDrive\\Desktop\\MLOps\\TextSummarizer'

## Data Ingestion Config - Data class

In [11]:
from dataclasses import dataclass
from pathlib import Path
import requests

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path


In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

### Configuration Updates

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config

## Components

In [13]:
import os
import urllib.request as request
import py7zr
from src.textSummarizer.logging import logger

In [14]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def downlaod_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"File is downloaded")
        else:
            logger.info(f"File already exits")

    def extract_zip_file(self):
        """
        Extracts the 7z file using py7zr library
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with py7zr.SevenZipFile(self.config.local_data_file, mode='r') as z:
            z.extractall(path=unzip_path)
            logger.info(f"7z file extracted to: {unzip_path}")

    


In [15]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)

data_ingestion.downlaod_file()
data_ingestion.extract_zip_file()

[2025-04-17 13:41:07,369]: INFO: common: yaml file: config\config.yaml loaded successfully:
[2025-04-17 13:41:07,370]: INFO: common: yaml file: params.yaml loaded successfully:
[2025-04-17 13:41:07,372]: INFO: common: created directory at: artifacts:
[2025-04-17 13:41:07,373]: INFO: common: created directory at: artifacts/data_ingestion:
[2025-04-17 13:41:08,950]: INFO: 78092507: File is downloaded:
[2025-04-17 13:41:09,139]: INFO: 78092507: 7z file extracted to: artifacts/data_ingestion:
